### Домашняя работа 2
Зенина Алёна БКЛ-201

In [1]:
import zipfile
import wget
import gensim
import numpy as np
from numpy import dot
from numpy. linalg import norm
import matplotlib.pyplot as plt
import scipy
import scipy.cluster.hierarchy as hcluster
from sklearn.cluster import KMeans

In [2]:
model_url = 'http://vectors.nlpl.eu/repository/20/220.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

100% [..................................................] 638171816 / 638171816

In [3]:
with open("HW2_igrat'.txt", "r", encoding="utf-8") as f:
    data = f.read().split('\n')[:-1]

In [4]:
def bag_of_words(string):
    bag = []
    for chunk in data:
        x = chunk.split('\t')
        y = x[0].split('_')
        for word in y:
            if word not in bag:
                bag.append(word)
    bag[0] += '_VERB'
    for i in range(1, len(bag)):
        bag[i] += '_NOUN'
    return bag

In [5]:
words = bag_of_words(data)
print(words)

['играть_VERB', 'роль_NOUN', 'музыка_NOUN', 'ребенок_NOUN', 'оркестр_NOUN', 'актер_NOUN', 'человек_NOUN', 'время_NOUN', 'карта_NOUN', 'музыкант_NOUN', 'солнце_NOUN', 'дитя_NOUN', 'спектакль_NOUN', 'комедия_NOUN', 'жизнь_NOUN', 'игра_NOUN', 'артист_NOUN', 'мальчик_NOUN', 'свадьба_NOUN', 'женщина_NOUN', 'день_NOUN', 'песня_NOUN', 'девочка_NOUN', 'вечер_NOUN', 'пьеса_NOUN', 'вальс_NOUN', 'команда_NOUN', 'улыбка_NOUN', 'ребята_NOUN', 'актриса_NOUN', 'ветер_NOUN', 'мальчишка_NOUN', 'шахматы_NOUN', 'гамлет_NOUN', 'возможность_NOUN', 'раз_NOUN', 'песнь_NOUN', 'год_NOUN', 'румянец_NOUN', 'рука_NOUN', 'отец_NOUN', 'россия_NOUN', 'ночь_NOUN', 'желание_NOUN', 'футбол_NOUN', 'сцена_NOUN', 'судьба_NOUN', 'мужчина_NOUN', 'театр_NOUN', 'мать_NOUN', 'девушка_NOUN', 'свет_NOUN', 'джаз_NOUN', 'ребятишки_NOUN', 'скрипка_NOUN', 'деньги_NOUN', 'гармонь_NOUN', 'шопен_NOUN', 'прятки_NOUN', 'лицо_NOUN', 'луч_NOUN', 'гость_NOUN', 'король_NOUN', 'мама_NOUN', 'старик_NOUN', 'трубач_NOUN', 'моцарт_NOUN', 'час_NOU

Слово "карта" встречается два раза, но так как модель к омонимии не чувствительная, объединяю

In [6]:
vectores = []
for word in words:
    if word in model:
        vectores.append(model[word])
    else:
        print('Увы, слова "%s" нет в модели!' % word)

In [7]:
add = []
for i in range(1, len(vectores)):
    add.append(vectores[0] + vectores[i])

In [8]:
def get_word(array):
    for i in range(len(add)):
        if np.array_equal(add[i], array):
            return 'играть_' + words[i + 1][:-5]

### Кластеризация методом иерархической кластеризации

In [9]:
X = np.array(add)
print(X.shape)

(112, 300)


In [10]:
def real_clusters(array):
    res = {}
    for number in set(array):   
        count = []
        for i in range(len(array)):
            if array[i] == number:
                       count.append(add[i])
        if len(count) > 2:
            res[number] = count
    return res            

Оптимальное значение для t (в котором максимально количество кластеров, содержащие больше 2 элементов) находится где-то в пределах от 0.5 (меньше 0 кластеров) до 1 (больше значение, меньше кластеров). с помощью функции выясняю:

In [11]:
Z = hcluster.linkage(X)
opt_i = 0
max_num = 0
for i in range(50, 101, 1):
    clusters = hcluster.fcluster(Z, i/100)
    if len(real_clusters(clusters)) > max_num:
        max_num = len(real_clusters(clusters))
        opt_i = i
        
print('Оптимальное значение: ', opt_i/100, '\nМаксимальное количество кластеров, размером больше 2: ', max_num)

Оптимальное значение:  0.71 
Максимальное количество кластеров, размером больше 2:  7


In [12]:
clusters = hcluster.fcluster(Z, 0.71)
print('Общее количество кластеров: %d' % len(set(clusters)), clusters)

Общее количество кластеров: 79 [75 21 23 36  4 59 62 68 24  2  9  5  5 54 77  4 12 42  7  3 31 12  3  5
 17 74  6 13  4 57 13 29  9 69 58 22 79  6 50 10  9  3 48 49 33 38  7 46
 10 14  2 21 13 16 61  9  9  9 60  2 32 71 10 11  9  9 41 52 35 45 27 65
 20 64 70 43 73 72 10 55 25 47  9  1 18 40  9 39 44 28 63 53 78  8 34 76
  8 67 15 30 19 11 66  9 51  1  9  3 37  9 56 26]


In [13]:
def cos_sim(a, b):
    return dot(a, b)/(norm(a)*norm(b))

In [14]:
def center(xclusters):
    res = {}
    for key in xclusters.keys():
        center = np.mean(xclusters[key], axis = 0)
        count = {}
        for vector in xclusters[key]:
            count[cos_sim(vector, center)] = get_word(vector)
        print(key, '\nСловосочетания:', count[sorted(count.keys())[-1]], count[sorted(count.keys())[-2]], count[sorted(count.keys())[-3]])         

In [15]:
center(real_clusters(clusters))

2 
Словосочетания: играть_луч играть_свет играть_солнце
3 
Словосочетания: играть_утро играть_вечер играть_ночь
4 
Словосочетания: играть_актер играть_актриса играть_артист
5 
Словосочетания: играть_пьеса играть_спектакль играть_комедия
9 
Словосочетания: играть_шопен играть_молчанка играть_мольер
10 
Словосочетания: играть_мать играть_отец играть_жена
13 
Словосочетания: играть_мальчишка играть_ребята играть_ребятишки


Поля вполне однородные, но их не очень много, немного смешиваются 5 и 9, где играть в молчанку и играть мольера имеют разные значения.
### Кластеризация методом К-средних
Аналогично ищем оптимальное значения для данного алгоритма

In [16]:
opt_i = 0
max_num = 0
for i in range(1, 50):
    kmeans = KMeans(n_clusters=i).fit(X)
    clusters = kmeans.labels_
    if len(real_clusters(clusters)) > max_num:
        max_num = len(real_clusters(clusters))
        opt_i = i
        
print('Оптимальное значение: ', opt_i, '\nМаксимальное количество кластеров, размером больше 2: ', max_num)

C:\Users\listv\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Оптимальное значение:  39 
Максимальное количество кластеров, размером больше 2:  16


In [18]:
kmeans = KMeans(n_clusters=39).fit(X)
center(real_clusters(kmeans.labels_))

1 
Словосочетания: играть_вальс играть_танец играть_песнь
2 
Словосочетания: играть_старик играть_старуха играть_дама
3 
Словосочетания: играть_актер играть_артист играть_актриса
6 
Словосочетания: играть_теннис играть_футбол играть_шахматы
9 
Словосочетания: играть_гамлет играть_пьеса играть_спектакль
10 
Словосочетания: играть_луч играть_свет играть_солнце
11 
Словосочетания: играть_утро играть_вечер играть_ночь
12 
Словосочетания: играть_мать играть_отец играть_жена
13 
Словосочетания: играть_шопен играть_молчанка играть_мольер
14 
Словосочетания: играть_фактор играть_процесс играть_значение
16 
Словосочетания: играть_музыка играть_джаз играть_песня
19 
Словосочетания: играть_любовь играть_сердце играть_жизнь
20 
Словосочетания: играть_женщина играть_мужчина играть_человек
29 
Словосочетания: играть_девочка играть_мальчик играть_мальчишка
33 
Словосочетания: играть_улыбка играть_глаз играть_румянец


Так как полей получилось больше, результаты получились более ровные, выделились кластеры с переносными значениями (14, 33), но появились дублирующие друг друга по смыслу, как группы 20 и 12